# NDVI Analysis with Landsat 8 using OpenGeo (GEE Style)

This notebook demonstrates how to perform NDVI analysis using the `opengeo` package. The API is designed to feel like Google Earth Engine (GEE) but runs locally using `xarray`, `stackstac`, and `leafmap`.

## 0. Install OpenGeo (if needed)
If you haven't installed OpenGeo yet, you can do so from the root directory or using the wheel file.

In [1]:
# Option 1: Install from source (recommended for local development)
# !pip install -e ../..

# Option 2: Install from wheel (adjust path if needed)
# !pip install ../../dist/opengeo-0.0.1-py3-none-any.whl

## 1. Initialize OpenGeo

OpenGeo can be initialized with various STAC API endpoints. Microsoft Planetary Computer is recommended for high-quality Landsat 8 data.

In [2]:
import opengeo as og
import os

# Enable no-sign-request for public data access
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

og.Initialize("MICROSOFT")

C:\Users\pulak\anaconda3\envs\maps\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


OpenGeo initialized with STAC API: https://planetarycomputer.microsoft.com/api/stac/v1/


## 2. Define Region of Interest (ROI)

Define a rectangular region over Bangalore.

In [3]:
roi = og.Geometry.Rectangle(77.55, 12.90, 77.65, 13.00)
print("ROI:", roi)

ROI: og.Geometry(POLYGON ((77.65 12.9, 77.65 13, 77.55 13, 77.55 12.9, 77.65 12.9)))


## 3. Load Landsat 8 Collection

Filter by date and bounds. We'll select the Red and NIR bands for NDVI calculation.

In [4]:
l8_col = og.ImageCollection("landsat-c2-l2") \
    .filterDate("2023-01-01", "2023-03-31") \
    .filterBounds(roi) \
    .filter(query={"platform": {"eq": "landsat-8"}}) \
    .select(["red", "nir08"])

print(f"Images found: {l8_col.size()}")

Images found: 5


## 4. Compute Median Composite and NDVI

Create a median composite to remove clouds and then calculate NDVI: `(NIR - Red) / (NIR + Red)`.

In [5]:
# Median composite (reducing over time)
image = l8_col.median(epsg=32643)

# Calculate NDVI
ndvi = image.normalizedDifference(["nir08", "red"]).rename("NDVI")

print("NDVI Image metadata:")
print(ndvi)

NDVI Image metadata:
og.Image({
  "type": "Image",
  "bands": [],
  "dtype": "float64",
  "shape": [
    367,
    360
  ],
  "crs": "EPSG:32643"
})


## 5. Visualize on Map

Display the NDVI results using an interactive map.

In [6]:
Map = og.Map()

vis_params = {
    'min': 0,
    'max': 0.8,
    'palette': 'red, yellow, green'
}

Map.addLayer(ndvi, vis_params, "Landsat 8 NDVI")
Map.centerObject(roi, zoom=12)
Map

Map(center=[12.9502105, 77.59984349999999], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

## 6. Zonal Statistics

Calculate the mean NDVI value for the entire region.

In [7]:
stats = ndvi.reduceRegion(
    reducer='mean', 
    geometry=roi, 
    scale=100
)
print("Mean NDVI across ROI:", stats)

Clip failed: No data found in bounds. Data variable: stackstac-fe16c1c21456ab3bb3fb458678044f1c


Mean NDVI across ROI: {'constant': 0.30835150434594366}
